In [ ]:
import torch
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler

model_id = "/data/duantong/mazipei/HuggingFace-Download-Accelerator/data/hf_cache/hub/models--stable-diffusion-v1-5--stable-diffusion-v1-5/snapshots/f03de327dd89b501a01da37fc5240cf4fdba85a1"
pipeline = DiffusionPipeline.from_pretrained(model_id, use_safetensors=True, torch_dtype=torch.float16, variant="fp16")
pipeline.to("cuda")
pipeline.scheduler = DPMSolverMultistepScheduler.from_config(pipeline.scheduler.config)

prompt = "portrait photo of a old warrior chief"

def get_inputs(batch_size=1):
    generator = [torch.Generator("cuda").manual_seed(i) for i in range(batch_size)]
    prompts = batch_size * [prompt]
    num_inference_steps = 20
    return {"prompt": prompts, "generator": generator, "num_inference_steps": num_inference_steps}

In [ ]:
from diffusers.utils import make_image_grid

images = pipeline(**get_inputs(16)).images
make_image_grid(images, 4, 4)

In [ ]:
# 由于 transformer 的注意力机制，显存占用很大，因此可以启用注意力切片串行化来减少显存的消耗
pipeline.enable_attention_slicing()